# a02. Push Scripts to Gitlab
## Push the Required Scripts Which Will be Used for Model Retraining and Scoring 

---

## Table of Content
1. [新增重新訓練程式碼 - retrain.py](#1.-新增重新訓練程式碼---retrain.py)<br>
2. [新增模型評分程式碼 - score.py](#2.-模型評分程式碼---score.py)<br>
3. [Push 到 Gitlab Repository](#3.-Push-到-Gitlab-Repository)<br>
3-1. 新增程式碼檔案<br>
3-2. 新增專案分支<br>
3-3. 新增merge request<br>
3-4. 接受merge request<br>

## Reference
1. [Python gitlab Package - Overview](https://github.com/python-gitlab/python-gitlab)
2. [Python gitlab Package - Getting Started](https://python-gitlab.readthedocs.io/en/stable/api-usage.html?highlight=get%20started)
3. [Gitlab Docs - API resources](https://docs.gitlab.com/ee/api/api_resources.html)

---
# 1. 新增重新訓練程式碼 - retrain.py
[Top](#Table-of-Content)

In [ ]:
from datetime import datetime, date
time_now = datetime.now().strftime("%Y%m%d-%H%M")

In [39]:
%%writefile ./retrain.py
import pandas
import sasctl
import pickle
import pkg_resources
from sasctl.services import model_repository
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split 

#00 setup
model_id = '43fb0ca8-77da-4a08-b1c9-23e9eb8cd49f'
data = pd.read_csv('https://support.sas.com/documentation/onlinedoc/viya/exampledatasets/hmeq.csv')
sas_session = sasctl.Session('10.249.6.39', 'modelDeveloper', 'demopw', verify_ssl = False)

#01 create new minor version
model_repository.create_model_version(model = model_id, minor = True)
current_model_version = model_repository.get_model(model_id)['modelVersionName'].replace('.', '_')


#02 delete all the model artifacts
model_repository.delete_model_contents(model = model_id)


#03 train a new model
y = data['BAD']
X = data.select_dtypes(exclude = ['object'])
del X['BAD']

X = X.fillna(0) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 2021)

model_gb = GradientBoostingClassifier()
model_gb.fit(X_train, y_train)


#04 upload the new model into MM
with open(f'./model_{current_model_version}.pkl', 'wb') as file:
    pickle.dump(model_gb, file)

myModel_pkl = open(f'./model_{current_model_version}.pkl', 'rb')
model_repository.add_model_content(model = model_id, file = myModel_pkl, name = f'./model_{current_model_version}.pkl')

Overwriting ./retrain.py


---
# 2. 模型評分程式碼 - score.py
[Top](#Table-of-Content)

In [2]:
%%writefile ./score.py
import pandas


#01 list to dataframe
input_from_user = list(X_test.iloc[1])
input_df = pandas.DataFrame([input_from_user], columns=('LOAN', 'MORTDUE', 'VALUE', 'YOJ', 'DEROG', 'DELINQ', 'CLAGE', 'NINQ', 'CLNO', 'DEBTINC'))


#02 in case, impute missing values 
input_df = input_df.fillna(0) 


#03 execute model inference
proba = model_gb.predict_proba(input_df)
proba_df = pandas.DataFrame(proba, columns=('P_bad', 'P_good'))                                    

Writing ./score.py


---
# 3. Push 到 Gitlab Repository
[Top](#Table-of-Content)

### 3-1. 新增程式碼檔案

In [ ]:
import gitlab

with open('./retrain.py', 'r') as my_file:
    file_content = my_file.read()

f = project.files.create(
    {'file_path': 'retrain.py',
     'branch': 'master',
     'content': file_content,
     'author_email': 'linyuhong2222@gmail.com',
     'author_name': "fkl",
     #'encoding': 'utf-8',
     'commit_message': 'Commit message here.'})



with open('./score.py', 'r') as my_file:
    file_content = my_file.read()

f = project.files.create(
    {'file_path': 'score.py',
     'branch': 'master',
     'content': file_content,
     'author_email': 'linyuhong2222@gmail.com',
     'author_name': "fkl",
     #'encoding': 'utf-8',
     'commit_message': 'Commit message here.'})

### 3-2. 新增專案分支

In [23]:
gl = gitlab.Gitlab(url = 'https://gitlab.com/', private_token = 'ynJH8ppLyiB_2L5sT2NY')

project = gl.projects.get('24272994')

branch = project.branches.create({'branch': 'branch' + , 'ref': 'master'})

data = {
    'branch': 'branch' + ,
    'commit_message': 'blah blah blah',
    'actions': [
        {
            'action': 'update',
            'file_path': 'retrain.py',
            'content': open('./retrain.py', 'r').read(),
        }
    ]
}

commit = project.commits.create(data)

# See https://docs.gitlab.com/ce/api/commits.html#create-a-commit-with-multiple-files-and-actions 
# for actions detail

### 3-3. 新增merge request

In [24]:
mr = project.mergerequests.create({
    'source_branch': 'ddd',
    'target_branch': 'master',
    'title': 'merge request from frank'})

# mr.delete()

mr.id

90887925

In [25]:
mrs = project.mergerequests.list(state='all', order_by='updated_at')
mrs

# state: state of the MR. It can be one of all, merged, opened or closed
# order_by: sort by created_at or updated_at

[<ProjectMergeRequest iid:3>,
 <ProjectMergeRequest iid:2>,
 <ProjectMergeRequest iid:1>]

### 3-4. 接受merge request

In [ ]:
mr.merge()

---